El **transfer learning** consiste en emplear las capas de convolución de un modelo ya entrenado sobre una basta cantidad de imágenes y sólamente entrenar las capas de la red neuronal.
A esto se le llama fine tuning
Algunas CNN preentrenadas que podemos emplear son:
| Modelo | Tamaño de entrada | Salida | Dataset entrenado | Tamaño del dataset |
|:-------|:-----------------:|:------:|:-----------------:|-------------------:|
| VGG16 | (224,224,3) | 1000 | 'imagenet' | 1281167 |
| ResNet50 | (224,224,3) | 1000 | 'imagenet' | 1281167 |
| MobileNetV2 | (224,224,3) | 1000 | 'imagenet' | 1281167 |
| EfficientNetB7 | (600,600,3) | 1000 | 'imagenet' | 1281167 |

Nosotros vamos a usar en este caso un dataset que es Covid19 Image Dataset que clasifica entre covid-19, neumonía y normal.
En su información podemos ver que solo tiene 317 imágenes.
Por eso mismo, vamos a tratar con estas pocas imágenes de conseguir una efectividad decente.

In [2]:
!pip install -q tensorflow==2.15.0 keras==2.15.0 tensorflow_hub==0.15.0

In [3]:
paths = ["/kaggle/input/covid19-image-dataset/Covid19-dataset/train/",
        "/kaggle/input/covid19-image-dataset/Covid19-dataset/test/"]

In [4]:
import os, glob, cv2, numpy as np, matplotlib.pyplot as plt

In [5]:
def imshow(img):
    fig, ax = plt.subplots(1, 1, figsize = (8,8))
    ax = imshow(img, cmap = 'gray')

In [6]:
possible_labels = os.listdir(paths[0])

In [7]:
possible_labels

['Normal', 'Viral Pneumonia', 'Covid']

In [8]:
data = []

for i, path in enumerate(paths):
    for label_int, label_string in enumerate(possible_labels):
        filenames = glob.glob(path + label_string + "/*.jpg") + glob.glob(path + label_string + "/*.jpeg") + glob.glob(path + label_string + "/*.png")
        for filename in filenames:
            img = cv2.imread(filename)
            data.append([label_int, cv2.resize(img, (224, 224))])
len(data)

317

In [9]:
import random
random.Random(0).shuffle(data)

In [10]:
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for i, sample in enumerate(data):
    label = sample[0]
    img = sample[1]
    if i <= 0.8 * len(data):
        x_train.append(img)
        y_train.append(label)
    elif i > 0.8 * len(data) and i <= 0.9 * len(data):
        x_val.append(img)
        y_val.append(label)
    else:
        x_test.append(img)
        y_test.append(label)

x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [11]:
import tensorflow as tf
y_trainOneHot = tf.one_hot(y_train, len(possible_labels))
y_valOneHot = tf.one_hot(y_val, len(possible_labels))
y_testOneHot = tf.one_hot(y_test, len(possible_labels))

2025-10-07 21:30:01.700398: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-07 21:30:01.700452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-07 21:30:01.702064: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

In [13]:
def cnn_model():
    model = Sequential()
    model.add(Convolution2D(32, (3,3), input_shape = (224,224,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Flatten())
    model.add(Dense(30, activation = 'relu'))
    model.add(Dense(3, activation = 'softmax'))
    model.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [14]:
model = cnn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 30)                11828190  
                                                                 
 dense_1 (Dense)             (None, 3)                 93        
                                                                 
Total params: 11829179 (45.12 MB)
Trainable params: 11829179 (45.12 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [15]:
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
checkpoint = ModelCheckpoint('checkpoint/model.{epoch:d}.h5', save_best_only = False, save_freq = 'epoch')
tensorboard_callback = tf.keras.callbacks.TensorBoard('logs/cnn_logs', histogram_freq = 1)

In [16]:
model.fit(x_train,
         y_trainOneHot,
         epochs = 40,
         batch_size = 100,
         validation_data = (x_val, y_valOneHot)
         )

Epoch 1/40
3/3 [==============================] - 5s 1s/step - loss: 8446007.0000 - accuracy: 0.4016 - val_loss: 1385.9751 - val_accuracy: 0.3438
Epoch 2/40
3/3 [==============================] - 4s 1s/step - loss: 38931320.0000 - accuracy: 0.3898 - val_loss: 335.2518 - val_accuracy: 0.3750
Epoch 3/40
3/3 [==============================] - 4s 1s/step - loss: 158.6428 - accuracy: 0.3504 - val_loss: 1.0985 - val_accuracy: 0.3750
Epoch 4/40
3/3 [==============================] - 4s 1s/step - loss: 1.0969 - accuracy: 0.4291 - val_loss: 1.0984 - val_accuracy: 0.3750
Epoch 5/40
3/3 [==============================] - 4s 1s/step - loss: 1.0965 - accuracy: 0.4291 - val_loss: 1.0983 - val_accuracy: 0.3750
Epoch 6/40
3/3 [==============================] - 4s 1s/step - loss: 1.0962 - accuracy: 0.4291 - val_loss: 1.0981 - val_accuracy: 0.3750
Epoch 7/40
3/3 [==============================] - 4s 1s/step - loss: 1.0958 - accuracy: 0.4291 - val_loss: 1.0980 - val_accuracy: 0.3750
Epoch 8/40
3/3 [=====

In [17]:
possible_labels[np.argmax(model.predict(x_test[0:1]))]

1/1 [==============================] - 0s 83ms/step


'Covid'

In [18]:
model.evaluate(x = x_test, y = y_testOneHot)

1/1 [==============================] - 0s 205ms/step - loss: 1.0741 - accuracy: 0.5161


[1.0741198062896729, 0.5161290168762207]

# **Método con transfer learning**
---

In [19]:
!pip install tensorflow_hub

In [20]:
import tensorflow_hub as hub

In [21]:
#Copiamos la url del modelo ya entrenado que vamos a usar
path_model = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5'

In [22]:
#Definimos nuestro modelo de transfer learning
def tfLearning_model():
    model = tf.keras.Sequential()
    #Ya no hacemos convoluciones ni flatten porque eso lo ha hecho ya el otro modelo
    #Aquí solo añadiremos ese modelo y le ponemos que no se entrene porque ya está entrenado
    model.add(hub.KerasLayer(path_model, trainable = False, input_shape = (224, 224, 3)))
    model.add(tf.keras.layers.Dense(30, activation = 'relu'))
    model.add(tf.keras.layers.Dense(3, activation = 'softmax'))
    model.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [24]:
model_tfLearning = tfLearning_model()

In [25]:
#Este resumen nos dice que entrenemos con pocas imágenes
model_tfLearning.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_4 (Dense)             (None, 30)                38430     
                                                                 
 dense_5 (Dense)             (None, 3)                 93        
                                                                 
Total params: 2296507 (8.76 MB)
Trainable params: 38523 (150.48 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [26]:
model_tfLearning.fit(x_train,
         y_trainOneHot,
         epochs = 40,
         batch_size = 100,
         validation_data = (x_val, y_valOneHot)
         )

Epoch 1/40
3/3 [==============================] - 12s 2s/step - loss: 1.2156 - accuracy: 0.3346 - val_loss: 1.1812 - val_accuracy: 0.3750
Epoch 2/40
3/3 [==============================] - 5s 2s/step - loss: 0.9902 - accuracy: 0.5354 - val_loss: 0.8576 - val_accuracy: 0.6250
Epoch 3/40
3/3 [==============================] - 5s 2s/step - loss: 0.8940 - accuracy: 0.6220 - val_loss: 0.8844 - val_accuracy: 0.5938
Epoch 4/40
3/3 [==============================] - 5s 2s/step - loss: 1.0170 - accuracy: 0.4685 - val_loss: 1.0872 - val_accuracy: 0.3750
Epoch 5/40
3/3 [==============================] - 5s 2s/step - loss: 1.0361 - accuracy: 0.4370 - val_loss: 0.7770 - val_accuracy: 0.6250
Epoch 6/40
3/3 [==============================] - 5s 2s/step - loss: 0.8471 - accuracy: 0.5906 - val_loss: 0.7751 - val_accuracy: 0.6875
Epoch 7/40
3/3 [==============================] - 6s 2s/step - loss: 0.8452 - accuracy: 0.6181 - val_loss: 0.7746 - val_accuracy: 0.7188
Epoch 8/40
3/3 [========================

In [30]:
#Evaluamos con el nuevo modelo
model_tfLearning.evaluate(x = x_test, y = y_testOneHot)

1/1 [==============================] - 1s 502ms/step - loss: 0.5066 - accuracy: 0.8065


[0.5065655708312988, 0.8064516186714172]